In [1]:
import os  #This allows access to environment variables
from dotenv import load_dotenv #this allows us to access the .env file
import spotipy # This is how you can connect to Spotify API
from spotipy.oauth2 import SpotifyOAuth #This is how to connect to Spotify API with OAuth
import pandas as pd #Pandas library for data manipulation
from datetime import datetime # used to convert some data
import time #used so that we can limit get requests

load_dotenv()

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=os.getenv("SPOTIPY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"),
    redirect_uri="http://127.0.0.1:8080/callback",
    scope="playlist-read-private playlist-read-collaborative user-library-read",
    open_browser=True
))

def get_playlist_tracks(playlist_id, playlist_name):
    """Get all tracks from a playlist"""
    tracks_data = []

    print(f"Fetching tracks from '{playlist_name[:40]}'...", end=" ")

    try:
        # Get tracks with pagination
        results = sp.playlist_items(playlist_id, limit=100)
        tracks = results['items']

        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])

        # Process each track
        for item in tracks:
            track = item.get('track')
            added_at = item.get('added_at')

            if track and track.get('id'):
                # Get all artist names
                artists = [artist['name'] for artist in track['artists']]
                artist_ids = [artist['id'] for artist in track['artists']]

                # Get genres from first artist
                try:
                    artist_info = sp.artist(artist_ids[0])
                    genres = ', '.join(artist_info.get('genres', []))
                except:
                    genres = ''

                track_info = {
                    'playlist_name': playlist_name,
                    'added_at': added_at,
                    'track_name': track['name'],
                    'artist': ', '.join(artists),
                    'artist_id': artist_ids[0] if artist_ids else '',
                    'album': track['album']['name'],
                    'album_type': track['album']['album_type'],
                    'release_date': track['album']['release_date'],
                    'duration_ms': track['duration_ms'],
                    'duration_min': round(track['duration_ms'] / 60000, 2),
                    'popularity': track['popularity'],
                    'is_explicit': track['explicit'],
                    'genres': genres,
                    'track_id': track['id'],
                    'track_url': track['external_urls']['spotify'],
                    'preview_url': track.get('preview_url', ''),
                }
                tracks_data.append(track_info)

        print(f"✓ {len(tracks_data)} tracks")
        return tracks_data

    except Exception as e:
        print(f"✗ Error: {e}")
        return []

# Main execution
try:
    # Get only the first 50 playlists
    print("Fetching your playlists...")
    results = sp.current_user_playlists(limit=50)
    all_playlists = results['items'][:60]  # Limit to exactly 50

    print(f"\nFound {len(all_playlists)} playlists (limited to 50)")
    print("="*60)

    # Show which playlists will be processed
    print("\nPlaylists to be processed:")
    for idx, playlist in enumerate(all_playlists, 1):
        print(f"{idx:2d}. {playlist['name'][:50]} ({playlist['tracks']['total']} tracks)")

    print("\n" + "="*60)
    print("Starting extraction...\n")

    # Process each playlist
    all_tracks = []
    successful_count = 0

    for idx, playlist in enumerate(all_playlists, 1):
        print(f"[{idx}/50] ", end="")

        tracks_data = get_playlist_tracks(playlist['id'], playlist['name'])

        if tracks_data:
            all_tracks.extend(tracks_data)
            successful_count += 1

        time.sleep(0.5)  # Rate limiting

    # Save to CSV
    if all_tracks:
        df = pd.DataFrame(all_tracks)

        # Generate filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"spotify_top50_playlists_{timestamp}.csv"

        df.to_csv(filename, index=False, encoding='utf-8-sig')

        print("\n" + "="*60)
        print(f"✓ SUCCESS!")
        print(f"  • Total tracks: {len(df):,}")
        print(f"  • Playlists processed: {successful_count}/50")
        print(f"  • Average tracks per playlist: {len(df)//successful_count if successful_count > 0 else 0}")
        print(f"  • Filename: '{filename}'")

        print(f"\nPlaylist breakdown:")
        playlist_counts = df['playlist_name'].value_counts()
        for playlist, count in playlist_counts.items():
            print(f"  • {playlist[:45]}: {count} tracks")

        print(f"\nData summary:")
        print(f"  • Unique tracks: {df['track_id'].nunique():,}")
        print(f"  • Unique artists: {df['artist'].nunique():,}")
        print(f"  • Average popularity: {df['popularity'].mean():.1f}")
        print(f"  • Explicit tracks: {df['is_explicit'].sum()} ({df['is_explicit'].mean()*100:.1f}%)")
        print(f"  • Average duration: {df['duration_min'].mean():.2f} min")

        print(f"\nSample data (first 10 tracks):")
        sample_cols = ['playlist_name', 'track_name', 'artist', 'popularity', 'duration_min']
        print(df[sample_cols].head(10).to_string(index=False))

        print(f"\nTop 10 most popular tracks:")
        top_tracks = df.nlargest(10, 'popularity')[['track_name', 'artist', 'popularity']]
        print(top_tracks.to_string(index=False))

    else:
        print("\n No tracks could be processed from any playlist")

except Exception as e:
    print(f"\n Error: {e}")
    import traceback
    traceback.print_exc()

Fetching your playlists...

Found 50 playlists (limited to 50)

Playlists to be processed:
 1. Studies (8 tracks)
 2. My Playlist #117 (17 tracks)
 3. Queue (33 tracks)
 4. The Most Beautiful Piano Pieces (137 tracks)
 5. Mind is a playground (6 tracks)
 6. Русские песни 🇷🇺🪆❤️ (56 tracks)
 7. Oldies Station (10 tracks)
 8. My Playlist #113 (0 tracks)
 9. Elise work out  (21 tracks)
10. Elise’s Playlist (324 tracks)
11. Yellow line (8 tracks)
12. My Playlist #109 (6 tracks)
13. The Shoo me playlist (35 tracks)
14. Psychological warfare (16 tracks)
15. Some Christmas  (21 tracks)
16. GOOBA (13 tracks)
17. Break up with myself (171 tracks)
18. The shoo me song (0 tracks)
19. First dance song ideas (19 tracks)
20. Reception dance party (17 tracks)
21. 👦 🌊  (89 tracks)
22. 80s Hits - Best of the 80s (81 tracks)
23. May STD 23 (75 tracks)
24. Ghost (8 tracks)
25. A love story (2) (25 tracks)
26. Lyrically beautiful (19 tracks)
27. This ones for Elise (21 tracks)
28. No reason to be sad other